## Formatting Output and Speaking for Claude

*(Coding along with the [Anthropic's Prompt Engineering Interactive Tutorial](https://github.com/anthropics/courses/tree/master/prompt_engineering_interactive_tutorial/Anthropic%201P) which can be found on Anthropic's courses GitHub repo)*

### Basic setup

In [1]:
# https://github.com/anthropics/courses/blob/master/prompt_engineering_interactive_tutorial/Anthropic%201P/01_Basic_Prompt_Structure.ipynb
from anthropic import Anthropic
import pandas as pd
import re

anthropic_api_key = pd.read_csv("~/tmp/anthropic/anthropic-key-1.txt", sep=" ", header=None)[0][0]
print("Don't be a fool and sent your api key to github")

# instantiating the client
client = Anthropic(api_key=anthropic_api_key)
MODEL_NAME="claude-3-5-sonnet-20241022"

Don't be a fool and sent your api key to github


In [2]:
# helper function that sends a prompt to Claude and returns Claude's generated response
# new argument added for prefill text, with a default value of an empty string
def get_completion(prompt: str, system_prompt="", prefill=""):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        messages=[
          {"role": "user", "content": prompt},
          {"role": "assistant", "content": prefill}
        ]
    )
    return message.content[0].text

We can use XML tags to make your prompt clearer and more parseable to Claude, but we can also __ask Claude to use XML tags to make its output clearer__ and more easily understandable to humans.

#### Examples: `poem preamble problem`

- Asking Claude to skip the preamble entirely by telling Claude to put the poem in XML tags.
- Having the output in XML tags allows the end user to reliably get the poem and only the poem by writing a short program to extract the content between XML tags.

In [3]:
# Variable content
ANIMAL = "Rabbit"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Put it in  tags."

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print(PROMPT)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT))

--------------------------- Full prompt with variable substutions ---------------------------
Please write a haiku about Rabbit. Put it in  tags.

------------------------------------- Claude's response -------------------------------------
<haiku>
Soft cotton tail hops
Through dewy morning meadow
Whiskers twitch with joy
</haiku>


#### Extension: Putting the first XML tag in the assistant turn

"Speaking for Claude" or "prefilling Claude's response": We're putting the first XML tag in the assistant turn, basically telling Claude that it has already said something, and that it should continue from that point onward.

In [7]:
# Variable content
ANIMAL = "Cat"

# Prompt template with a placeholder for the variable content
# no need to ask to put it in tags if we provide the first tag with the assistant message
PROMPT = f"Please write a haiku about {ANIMAL}."

# Prefill for Claude's response
PREFILL = "<haiku_tag>"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN:")
print(PROMPT)
print("\nASSISTANT TURN:")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN:
Please write a haiku about Cat.

ASSISTANT TURN:
<haiku_tag>

------------------------------------- Claude's response -------------------------------------

Soft paws pad silent
Whiskers twitch in morning light
Napping in sunbeams
</haiku_tag>


#### Enforcing JSON output

If we want to enforce JSON output, we can prefill Claude's response with the opening bracket, `{}`.

In [9]:


# Variable content
ANIMAL = "Cat"

# Prompt template with a placeholder for the variable content
PROMPT = f"Please write a haiku about {ANIMAL}. Use JSON format with the keys as \"first_line\", \"second_line\", and \"third_line\"."

# Prefill for Claude's response
PREFILL = "{" # works with and without prefill

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN
Please write a haiku about Cat. Use JSON format with the keys as "first_line", "second_line", and "third_line".

ASSISTANT TURN


------------------------------------- Claude's response -------------------------------------
{
  "first_line": "Soft paws on window",
  "second_line": "Whiskers twitch in morning light",
  "third_line": "Napping in the sun"
}


#### Example of multiple input variables in the same prompt & output formatting specification

In [10]:
# example of multiple input variables in the same prompt AND output formatting specification
# all done using XML tags
# First input variable
EMAIL = "Hi Zack, just pinging you for a quick update on that prompt you were supposed to write."

# Second input variable
ADJECTIVE = "olde english"

# Prompt template with a placeholder for the variable content
PROMPT = f"Hey Claude. Here is an email: {EMAIL}. Make this email more {ADJECTIVE}. Write the new version in <{ADJECTIVE}_email> XML tags."

# Prefill for Claude's response (now as an f-string with a variable)
PREFILL = f"<{ADJECTIVE}_email>"

# Print Claude's response
print("--------------------------- Full prompt with variable substutions ---------------------------")
print("USER TURN")
print(PROMPT)
print("\nASSISTANT TURN")
print(PREFILL)
print("\n------------------------------------- Claude's response -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

--------------------------- Full prompt with variable substutions ---------------------------
USER TURN
Hey Claude. Here is an email: Hi Zack, just pinging you for a quick update on that prompt you were supposed to write.. Make this email more olde english. Write the new version in <olde english_email> XML tags.

ASSISTANT TURN
<olde english_email>

------------------------------------- Claude's response -------------------------------------

Dearest Zack,

I do hope this missive finds thee in good health and spirits. I write to thee on this fine day to humbly inquire about the status of yon prompt which thou were tasked to compose.

Pray tell, might thou grace me with news of its progress?

With warmest regards,
Thy faithful correspondent
</olde english_email>
